In [50]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import *
from math import pi
import numpy as np
import matplotlib.pyplot as plt

In [51]:
def apply_ctrl_z(qc:QuantumCircuit, q0:int, q1:int):
    qc.rz(pi/2,q1)
    qc.cx(q0,q1)
    qc.rz(-pi/2,q1)
    qc.cx(q0,q1)

    return

def apply_rx_pi_4(qc:QuantumCircuit, q0:int):
    qc.rz(pi/2,q0)
    qc.sx(q0)
    qc.rz(5*pi/4,q0)
    qc.sx(q0)
    qc.rz(pi/2,q0)

    return

def apply_rx_neg_pi_4(qc:QuantumCircuit, q0:int):
    qc.rz(pi/2,q0)
    qc.sx(q0)
    qc.rz(3*pi/4,q0)
    qc.sx(q0)
    qc.rz(pi/2,q0)

    return

def apply_ctrl_sx(qc:QuantumCircuit, q0:int, q1:int):
    apply_rx_pi_4(qc,q1)
    apply_ctrl_z(qc, q0, q1)
    apply_rx_neg_pi_4(qc,q1)
    apply_ctrl_z(qc, q0, q1)

def apply_ctrl_sx_dg(qc:QuantumCircuit, q0:int, q1:int):
    apply_ctrl_z(qc, q0, q1)
    apply_rx_neg_pi_4(qc,q1)
    apply_ctrl_z(qc, q0, q1)
    apply_rx_pi_4(qc,q1)

In [52]:
def generateBinaryStrings(ndim:int):
    res = ["0","1"]
    while len(res)<ndim:
        s = res.pop()
        res.insert(0,s+"1")
        res.insert(0,s+"0")

    return res

def getSymmetricPairs(bStringArray:list):
    pair_list = []
    tmp = []
    for s in bStringArray:
        tmp.append(s[::-1])
    d_tmp = bStringArray2DecArray(tmp)
    tmp2 = []
    
    for i in range(len(bStringArray)):
        if d_tmp[i]!=i and i not in tmp2:
            tmp2.append(i)
            tmp2.append(d_tmp[i])
    
    for i in range(len(tmp2)//2):
        pair_list.append([tmp2[2*i],tmp2[2*i+1]])
        
    return pair_list 

def bStringArray2DecArray(bArray:list):
    dArray =[]
    for s in bArray:
        l = len(s)
        res = 0
        for i in range(l):
            res += int(s[i])*2**(l-1-i)
        dArray.append(res)
    return dArray

def qiskitU2textU(unitary:np.ndarray):
    n,m=unitary.shape
    assert m==n
    exchange_pairs = getSymmetricPairs(generateBinaryStrings(n))
    for pair in exchange_pairs:
        reverse_pair = pair[::-1]
        unitary[pair] = unitary[reverse_pair]
        unitary[:,pair] = unitary[:,reverse_pair]
    return unitary

In [53]:
def debug_get_unitary(qc:QuantumCircuit):
    backend = Aer.get_backend("unitary_simulator")
    job = execute(qc, backend=backend)
    unitary = job.result().get_unitary()
    TextUnitary = qiskitU2textU(unitary)
    norm = np.linalg.norm(np.trace(unitary))

    return TextUnitary, unitary, norm

In [54]:
qc = QuantumCircuit(2)
qc.csx(0,1)
qc.draw()

q_0: ──■───
     ┌─┴──┐
q_1: ┤ √X ├
     └────┘

In [55]:
unitary,_,_ = debug_get_unitary(qc)
print(unitary)

[[1. +0.j  0. +0.j  0. +0.j  0. +0.j ]
 [0. +0.j  1. +0.j  0. +0.j  0. +0.j ]
 [0. +0.j  0. +0.j  0.5+0.5j 0.5-0.5j]
 [0. +0.j  0. +0.j  0.5-0.5j 0.5+0.5j]]
